In [1]:
%%html
<style>
    /* Jupyter */
    .rendered_html tr, .rendered_html th, .rendered_html td {
        text-align: left; 
    }
</style>

# Holt-Winters Method for Time Series Forecasting

## Learning Objectives
- Time Series
- Forecasting Accuracy
- Simple forecasting methods
- Exponential Smoothing
- Level, Trend, Seasonality
- Holt-Winters Method
- Python Implementation of HW's
- Initialising trend, slope, alpha, beta, gamma

(Add links to each section here)

In this notebook we'll cover an introduction to time series, key terms used to describe the important features of time series and how to forecast time series using the Holt-Winters method.

## Time Series

A time series is a sample of measurements of some interesting quantity taken repeatedly over a sustained period of time. It is mathematically represented as a set of vectors x(t),t = 0,1,2,... where t represents the time elapse and the variable x(t) is treated as a random variable. A defining characteristic of a time series is that it is a list of observations where the ordering matters so the data is not necessarily independent and identically distributed. Changing the order of the datapoints could change the meaning of the data. 

Time series appear in many different disciplines in the real world such as Economics, Epidemiology, Social Sciences, Physical Sciences etc. 

Let's have a look at an explicit example of a time series. The following time series dataset is taken from the NASA website and describes the increase in monthly CO2 levels (parts per million) since 2005. 

Insert code showing table preview of dataset and then time plot of data set.

Add more examples maybe...

## Forecasting
The rest of this notebook will be focusing on time series forecasting and building up to how to use the Holt-Winters method for time series forecasting in particular. But first what is the difference between the terms time series analysis and time series forecasting? Time series analysis is a form of descriptive modeling, this means that someone conducting time series analysis will be looking at a dataset to identify trends and seasonal patterns in the hostorical data, fitting mathematical models to capture the underlying nature of the process generating the data etc. Time series forecasting is a form of predictive modelling with the goal to predict a future value at a particular point in time based on the values we do know.

### Forecasting accuracy - Some notation and terminology:

In a time series the values we do know are referred to as observed values and the values we are trying to forecast are referred to as expected values. In general, we use the notation $\hat{y}$ to denote expected values. 

For example, if we have a series that looks like [2,4,6,8,10], we might forecast the next value of this series to be 12. Using this terminology and notation, the observed values are $y_1=2$, $y_2=4$, $y_3=6$, $y_4=8$, $y_5=10$ i.e the observed series is [2,4,6,8,10] and the next expected value is $\hat{y_6}=12$.

It's important to have some metrics to evaluate the accuracy of our forecasts. 

The error is the difference between an observed value and its forecast. Given a training dataset {$y_{1},\dots, y_{T}$} and a test dataset {$y_{T+1}, y_{T+2},\dots$}, the error of a forecast at a given time interval T+h is denoted as $e_{T+h}=y_{T+h} - \hat{y_{T+h}}$. 

As the error can be positive or negative it is more helpful to use the absolute terms or as common convention square the error the value is always positive. The sum of squared errors (SSE) is given by $SSE = {\Sigma_{i=1}}^{i=n} ( y_{i} - \hat{y_{i}})^{2}$. The SSE measures the inexplained variability or discrepancy between the observed data and the forecasted data. Another common metric used is the mean squared error which is given by $MSE=\frac{1}{n}{\Sigma_{i=1}}^{i=n} ( y_{i} - \hat{y_{i}})^{2}$.

Note to self: explain where n comes from or change the notion in error definition to include n.

## Some simple forecasting methods

#### Naive Method

This is the simplest forecasting method. For naive forecasts we set all forecasts to be the same value as the last observed value. That is 

$\hat{y}_{T+1}=y_{T}$

For example, if we have a time series that looks like [14,20,18,17,24], then using the naive method the forecast for the next point would be 24. 

#### Average Method

This method is simply the expected value of the next datapoint is the arithmetic mean of all of the previous datapoints. That is

$\hat{y}_{T+1}=\frac{1}{T}\Sigma_{i=1}^{i=T} y_{i}$

For example, if we have a time series that looks like [19.2,17.8,15.1,14.3,15.0,16.7,15.2], then using the average method the forecast for the next point would be 16.2.

#### Moving Averages

An improvement over the taking average of all points is instead only taking the average of the n latest datapoints. In this method only the most recent values matter. In practise this forecasting method can be effective if the right choice of n is used. 

$\hat{y}_{T+1}=\frac{1}{n}\Sigma_{i=0}^{i=n-1} y_{T-i}$

#### Weighted Moving Averages

Often we want something in between the extremes of taking a naive forecast where only the most recent datapoint is considered and taking an average of the historical data. A weighted moving average is a moving average but within the window of n points each point is assigned a different weighting. Typically the most recent points are assigned a higher weight as these would be more relevant to the forecast being made. Note that the weights assigned must add to 1. 

$\hat{y}_{T+1}=\frac{1}{n}\Sigma_{i=1}^{i=n} w_{i} . y_{T+1-i}$

where $w_{1}, w_{2}, \cdots, w_{n}$ are weights to be assigned.

Note to self: Code up the implementation of each of these forecasting methods,

## Exponential Smoothing

Now let's consider a weighted average method where we consider all of the observed data points. We want the most recent observations to be weighted the highest whilst the weights decrease exponentially as the observations go further back into the past. This method of forecasting is called exponential smoothing as the weights decay exponentially as we continue to got back to observed points in time.

For example the set of weights used could look like:

$0.8, 0.8^2, 0.8^3, 0.8^4, 0.8^5, 0.8^6 \cdots$ or equivalently 

$0.8, 0.64, 0.512, 0.4096, 0.32768, 0.262144 \cdots$

The issue here is that these weights do not add up to 1 (using geometric series the sum of the weights in the example above approaches 4).

To solve this issue exponential smoothing can be encapsulated in conscise and elegant formula:

$\hat{y_{x+1}}$ = $\alpha \cdot y_{x}+(1-\alpha) \cdot \hat{y_{x}}$

In a way you can consider exponential smoothing to be a weighted average of two different terms $y_{x}$ and $\hat{y_{x}}$, also now the weights $\alpha$ and $(1-\alpha)$ sum to 1. We can convince ourselves that this succint formula really is equivalent to exponential smoothing by substituting a couple of terms in:

$\hat{y_{x+1}}$ = $\alpha \cdot y_{x}+(1-\alpha) \cdot \hat{y_{x}}$

$ = \alpha \cdot y_{x}+(1-\alpha) \cdot \lbrack \alpha \cdot y_{x-1}+(1-\alpha) \cdot \hat{y_{x-1}}\rbrack$

$ = \alpha \cdot y_{x}+\alpha \cdot (1-\alpha) \cdot y_{x-1}+(1-\alpha)^{2} \cdot \hat{y_{x-1}}$

$\vdots$

$ = \alpha \cdot y_{x}+\alpha(1-\alpha) \cdot y_{x-1} + \alpha(1-\alpha)^{2} \cdot y_{x-2}+ \alpha(1-\alpha)^{3} \cdot y_{x-3}+ \alpha(1-\alpha)^{4} \cdot y_{x-4}+\dots $

By subsituting in more terms recursively we can now see that going back to the beginning of the series (or infinitely backwards), the weights are generated by multiplying $(1-\alpha)$. This is the same as exponential smoothing but now we have a nicer way to represent this.

The parameter $\alpha$ is referred to as the smoothing factor and can take values $0 \leq \alpha \leq 1$. A large value of $\alpha$ gives more weight to recent changes, the smaller the value of $\alpha$ more distant points are taken into consideration when making the focus. In a way the larger the value of $\alpha$ the quicker the method "forgets" past observations.

In theory a time series can go back infinitely but in practise when forecasting we can choose a point in time to start our time series. 
    
So far all of the forecasting methods discussed can only be used to forecast a single datapoint into the future. The rest of the notebook will take a look at how building on exponential smoothing we can get to a more useful model that can be used to predict further into the future and take into account other characteristics of the time series we are trying to forecast. 

## Holt-Winters Method for Forecasting

#### Level, Trend and Seasonality
Level of a time series is the expected value/avergae value of time time series. Note: As the Holt-Winters method for forecasting uses the level as part of the calculation, it is referred to as $l$ instead of $\hat{y}$.

The trend is the slope of long-term increase or decrease in the data. Trend is denoted by $b$ and is given by $b=y_{x} - y_{x-1}$. (Note to self: check this formula, doe sit only apply to linear case?)

If a time series repeats at a fixed frequency then this intereval is known as it's season. The season length is the number of data points within a season, this is denoted by L.

Note to self: Add code to show examples of all three features.

### Holt-Winters Method

The Holt-Winters model (also referred to as triple exponential smoothing) predicts a current or future value by computing the combined effects of these three influences. Note that non-seasonal series cannot be forecasted using the Holt-Winters method.

The idea behind the Holt-Winters model is to apply exponential smoothing to the level, trend and seasonal components. The smoothing is applied across the seasons e.g. the 2nd point of the season would be exponentially smoothed with the 2nd point of the second season, the 2nd point from the third season etc.

#### Holts-Winters Method Formulas:

Level: $l_{t} =\alpha(y_{t}- s_{t-L})+(1-\alpha)(\ell_{t-1}+b_{t-1})$

Trend: $b_{t}=\beta(\ell_{t}-\ell_{t-1})+(1-\beta)b_{t-1}$

Seasonal: $s_{t}=\gamma (y_{t}-\ell_{t})+(1-\gamma)s_{t-L}$

Forecast: $\hat{y_{t+m}}=\ell_{t}+mb_{t} + s_{t-L+1+(m-1) mod L} $

where 

$\alpha=$ smoothing paramenter for level component

$\beta=$ smoothing parameter for trend component

$\gamma=$ smoothing parameter for seasonal component

$m =$ time index being forecasted (integer) 

$L=$ season length

$\ell_{t} = $level component at time t

$b_{t}= $trend component at time t

$s_{t}=$ seasonal component at time t

The time index t+m can take any integer value meaning we can now forecast any number of time steps into the future!!! 

## Initialising Holt-Winters

#### Initial Trend
Initial trend for a time series with a seasonal component can be found be take the average of the trend averages across each season. i.e. 

$b_{0}=\frac{1}{L}(\frac{{y_{L+1}} − {y_{1}}}{L}+\frac{{y_{L+2}} − {y_{2}}}{L}+\cdots+\frac{{y_{L+L}} − {y_{L}}}{L})$

Include python implementation of initial trend here.

The code below computes the intial seasonal component. For more information on the mathematics behind this see the following link (add hyperlink here).

Include python implementation of initial seasonality here.

### Python implementation: Final Algorithm

Include code here for final holt winters algorithm

Add code to generate plot of time series observed and then plot with forecast generated by Holt Winters.

### Initialising alpha, beta, gamma

The simplest way to optimize the values of alpha, beta, gamma is to run the Holt-Winters method on a set of know observed values and choose the parameters which minimize the SSE. Trial and error should give a reasonable estimate for alpha, beta and gamma. A better way to optimise for alpha, beta, gamma is to use the Nelder - Mead algorithm Add links to further reading here.

## Summary
- Include summary of key takeaways
- Include challenges